Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Classification using Automated ML

In this example we use Azure ML's Automated ML functionality to improve on the classifier we built earlier. Automated ML handles the task of building many models from a wide variety of algorithms and choosing a good set of hyper-parameters for them. We then select best the model (or one that meets our criteria) and deploy it as a web service.

## Load and prepare experiment

In [4]:
%run "../includes/mnt_blob"

As part of the setup we have already created an AML workspace. Let's load the workspace and create an experiment.

In [6]:
import json
import logging
import os
import random

from matplotlib import pyplot as plt
from matplotlib.pyplot import imshow
import pandas as pd
from sklearn import datasets

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.train.automl.run import AutoMLRun

We load the workspace directly from the config file we created in the early part of the course.

In [8]:
config_path = '/dbfs/tmp/aml_config'
ws = Workspace.from_config(path=os.path.join(config_path, 'config.json'))

experiment_name =  'pred-maint-automl' # choose a name for experiment
project_folder = '.' # project folder

experiment=Experiment(ws, experiment_name)

output = {}
output['SDK version'] = azureml.core.VERSION
output['Subscription ID'] = ws.subscription_id
output['Workspace'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Project Directory'] = project_folder
output['Experiment Name'] = experiment.name
pd.set_option('display.max_colwidth', -1)
pd.DataFrame(data=output, index=['']).T

Opt in for diagnostics for better experience, quality, and security of future releases:

In [10]:
from azureml.telemetry import set_diagnostics_collection
set_diagnostics_collection(send_diagnostics=True)

## Instantiate config file

We now instantiate a `AutoMLConfig` object. This defines the settings and data used to run the experiment.

|Property|Description|
|-|-|
|**task**|classification or regression|
|**primary_metric**|This is the metric that you want to optimize.<br> Classification supports the following primary metrics <br><i>accuracy</i><br><i>AUC_weighted</i><br><i>balanced_accuracy</i><br><i>average_precision_score_weighted</i><br><i>precision_score_weighted</i>|
|**max_time_sec**|Time limit in seconds for each iterations|
|**iterations**|Number of iterations. In each iteration Auto ML trains the data with a specific pipeline|
|**n_cross_validations**|Number of cross validation splits|
|**X**|(sparse) array-like, shape = [n_samples, n_features]|
|**y**|(sparse) array-like, shape = [n_samples, ], [n_samples, n_classes]<br>Multi-class targets. An indicator matrix turns on multilabel classification.  This should be an array of integers. |
|**path**|Relative path to the project folder.  AutoML stores configuration files for the experiment under this folder. You can specify a new empty folder. |

In [13]:
import azureml.core

print("SDK Version:", azureml.core.VERSION)

In [14]:
filepath = '/dbfs/mnt/data/telemetry/'

X_train = pd.read_csv(filepath + 'X_train.csv', header=0)
X_test = pd.read_csv(filepath + 'X_test.csv', header=0)
y_train = pd.read_csv(filepath + 'y_train.csv', header=0)
y_test = pd.read_csv(filepath + 'y_test.csv', header=0)

In [15]:
X_train.head()

In [16]:
y_train.head()

Here are the metrics we can choose to optimize our model over.

In [18]:
azureml.train.automl.constants.Metric.CLASSIFICATION_PRIMARY_SET

We now set up a configuration file for the automated ML training experiment. It contains details for how the experiment should run.

In [20]:
automl_config = AutoMLConfig(task='classification', 
                             preprocess=False,
                             name=experiment_name,
                             debug_log='automl_errors.log',
                             primary_metric='AUC_weighted',
                             iteration_timeout_minutes=5,
                             iterations=10,
                             verbosity=logging.INFO,
                             X = X_train.values, # we convert from pandas to numpy arrays using .vaules
                             y = y_train.values[:, 0], # we convert from pandas to numpy arrays using .vaules
                             X_valid=X_test.values,
                             y_valid=y_test.values[:, 0],
                             path=project_folder)

## Run training experiment

You can call the submit method on the experiment object and pass the run configuration. For Local runs the execution is synchronous. Depending on the data and number of iterations this can run for while.
You will see the currently running iterations printing to the console.

In [23]:
local_run = experiment.submit(automl_config, show_output = True)

#### Portal URL for Monitoring Runs

The following will provide a link to the web interface to explore individual run details and status.

In [25]:
displayHTML("<a href={} target='_blank'>Your experiment in Azure Portal: {}</a>".format(local_run.get_portal_url(), local_run.id))

## Retrieve the Best Model

Below we select the best pipeline from our iterations. The *get_output* method on automl_classifier returns the best run and the fitted model for the last *fit* invocation. There are overloads on *get_output* that allow you to retrieve the best run and fitted model for *any* logged metric or a particular *iteration*.

In [27]:
best_run, fitted_model = local_run.get_output()

In [28]:
fitted_model

We can see from the above results that `StandardScalerWrapper` was used to scale the features and a `LightGBMClassifier` was chosen as the best model based on the metric we defined. This of course does NOT automatically also make it the best model in production, but choosing the right model for production is beyond the scope of this course so we will not address it here.

### Hands-on lab

Run the following cell and go to the link provided under Details Page. This links will take us to the Azure portal. Examine the content of the page. Can you find what resource group this resource is under? What kind of resource is it?

In [32]:
displayHTML("<a href={} target='_blank'>Your experiment in Azure Portal: {}</a>".format(best_run.get_portal_url(), best_run.id))

In addition to choosing a good algorithm, the experiment also tuned hyper-parameters. So our model didn't just run with the default hyper-parameter values. Find out how we can get the chosen hyper-parameters from the `fitted_model` object. Describe the hyper-parameters you see. Which ones do you think are the most critical ones?

In [34]:
# write solution here

### End of lab

We can run the following code to get the hyper-parameters for the chosen model.

In [37]:
scaler_params = fitted_model.steps[0][1].get_params()
scaler_params

In [38]:
model_params = fitted_model.steps[1][1].get_params()
model_params

## Score and evaluate the chosen model

Let's now pick the best model returned by the experiment and use it to get predictions for the test data. This is simply done by replacing `manual_model.predict` with `fitted_model.predict`.

In [41]:
y_pred = fitted_model.predict(X_test)

We should get the same confusion matrix we did in the section above.

In [43]:
confusion_matrix(y_test.values[:, 0], y_pred)

We use `classification_report` to automatically calculate precision, recall, and the F-1 score from the confusion matrix above.

In [45]:
cl_report = classification_report(y_test.values[:, 0], y_pred)
print(cl_report)

The AUC is just the area under the ROC curve shown here:

In [47]:
from sklearn.metrics import auc, roc_curve
fpr, tpr, thresholds = roc_curve(y_test.values[:, 0], y_pred)
roc_auc = auc(fpr, tpr)

import matplotlib.pyplot as plt
plt.plot(fpr, tpr, 'b', label = 'AUC = {0:.2f}'.format(roc_auc))
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()
display()

## Register fitted model for deployment

Now that we have a model we're happy with, we register it to our Azure ML account. This will be the first step toward model management and deployment, which we cover in the next Notebook. Registered models can also be loaded into other workspaces.

In [50]:
description = 'automated ML PdM (predict y_1)'
tags = None
model = local_run.register_model(description=description, tags=tags)
local_run.model_id # Use this id to deploy the model as a web service in Azur

Optionally, we can also take the model and save it on disk as a pickle file, as shown here:

In [52]:
from sklearn.externals import joblib
joblib.dump(value=fitted_model, filename='model.pkl')

### Hands-on lab

<div class="alert alert-info">
So far we used automated ML to automatically train a classification model for only one of the four failure types, namely `y_1`. Write a loop to use automated ML to train, score and evaluate and finally register a classification model for `y_2`, `y_3` and `y_4`. In each case, compare the evaluation metrics we obtain to the ones we obtained in the previous Notebook when we hand-trained a model.
</div>

In [55]:
all_output = {'y_1': {'local_run': local_run, 'model': model, 'cl_report': cl_report}}

for i in range(1, 4): ## loop over each target class
    print("Running automl on y_" + str(i+1))
    ## YOUR CODE GOES HERE (modify the config file to point to the right target class)
    ## YOUR CODE GOES HERE (submit the automated ML experiment)
    ## YOUR CODE GOES HERE (extract the best model)
    ## YOUR CODE GOES HERE (predict on the test data)
    ## YOUR CODE GOES HERE (find the classification report for the best model and add it to y_report)
    ## YOUR CODE GOES HERE (add results to all_output)
    ## YOUR CODE GOES HERE (register the final model)

Here's the solution:

In [57]:
all_output = {'y_1': {'local_run': local_run, 'model': model, 'cl_report': cl_report}}

for i in range(1, 4): ## loop over each target class
    print("Running automl on y_" + str(i+1))
    automl_config.fit_params.update({'y': y_train.values[:, i]})
    local_run = experiment.submit(automl_config, show_output=True)
    best_run, fitted_model = local_run.get_output()
    y_pred = fitted_model.predict(X_test)
    cl_report = classification_report(y_test.values[:, i], y_pred)
    print(cl_report)
    all_output['y_' + str(i+1)] = {'local_run': local_run, 'cl_report': cl_report}
    description = 'autometed ML PdM (predict y_{})'.format(str(i+1))
    model = local_run.register_model(description=description, tags=None)

<div class="alert alert-info">
NOTE: We can use automated ML to directly train a multi-class classification model instead of training many binary classification models. We will explore that in another Notebook.
</div>

### End of lab

# The end

Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.